In [8]:
import boto3
import datetime
from botocore.session import get_session
from botocore.credentials import RefreshableCredentials
import faiss
from langchain.embeddings import HuggingFaceBgeEmbeddings
import os
import json
from langchain.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.docstore.document import Document
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain.retrievers import BM25Retriever, EnsembleRetriever

In [31]:
!pip install -U rank_bm25

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [9]:
# ARN of Role A to assume  
role_to_assume = 'arn:aws:iam::605134468121:role/BedrockCrossAccount'

def get_credentials():
    sts_client = boto3.client('sts')
    assumed_role = sts_client.assume_role(
        RoleArn=role_to_assume,
        RoleSessionName='cross-account-session',
        # Don't set DurationSeconds when role chaining
    )
    return {
        'access_key': assumed_role['Credentials']['AccessKeyId'],
        'secret_key': assumed_role['Credentials']['SecretAccessKey'],
        'token': assumed_role['Credentials']['SessionToken'],
        'expiry_time': assumed_role['Credentials']['Expiration'].isoformat()
    }

session = get_session()
refresh_creds = RefreshableCredentials.create_from_metadata(
    metadata=get_credentials(),
    refresh_using=get_credentials,
    method='sts-assume-role'
)

# Create a new session with refreshable credentials
session._credentials = refresh_creds
boto3_session = boto3.Session(botocore_session=session)

In [10]:
region: str = "us-west-2"

In [11]:
import sys
!{sys.executable} -m pip install langchain_aws


In [12]:
from langchain_aws import ChatBedrockConverse
import boto3

# ---- ⚠️ Update region for your AWS setup ⚠️ ----
bedrock_client = boto3_session.client("bedrock-runtime",
                              region_name=region)

In [13]:
llm = ChatBedrockConverse(
    client=bedrock_client,
    model_id="us.amazon.nova-micro-v1:0",
)


In [16]:
faiss_index_path = 'faiss_index/faiss_index.index'
faiss_index = faiss.read_index(faiss_index_path)
embeddings_model = HuggingFaceBgeEmbeddings(
    model_name="BAAI/bge-small-en",
    model_kwargs={"device": "cpu"},
    encode_kwargs={"normalize_embeddings": True}
)

with open(os.path.join('faiss_index', 'metadata.json'), 'r') as f:
    metadata = json.load(f)


/home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/_distutils_hack/__init__.py:15: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
/home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/_distutils_hack/__init__.py:30: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(
2025-02-19 04:40:35.700659: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] 

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/90.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [17]:
docstore = {}
for i, meta in enumerate(metadata):
    doc = Document(page_content = meta.get("cotent", "A"), metadata = meta)
    docstore[i] = doc
    
doc = InMemoryDocstore(docstore)

In [18]:
#docstore = {metadata["id"]: metadata["subreddit"] for meta in metadata}
index_to_docstore_id = {i: doc_id for i, doc_id in enumerate(docstore.keys())}
#{i: meta["id"] for i, meta in enumerate(metadata)}

vectorstore = FAISS(
    embeddings_model.embed_query,
    index=faiss_index,
    docstore=doc,
    index_to_docstore_id=index_to_docstore_id
)



`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.


In [19]:
retriever = vectorstore.as_retriever(search_kwargs={'k': 4})

In [20]:
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(llm, prompt)

rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [21]:
results1 = rag_chain.invoke(
    {"input": "Compare and contrast coffee shops in New York and DC."}
)
output1 = {"question" : results1["input"], 
           "context": [r.metadata for r in results1["context"]], 
          "answer": results1["answer"]}

#print(results1["context"][0].metadata)
with open("problem3_task1.txt", "w") as f:
    json.dump(output1, f, indent=4)

In [22]:
retriever2 = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"filter": lambda doc: doc.get("num_comments") is not None and doc.get("num_comments") > 10
})

rag_chain2 = create_retrieval_chain(retriever2, question_answer_chain)

In [23]:
results2 = rag_chain2.invoke(
    {"input": "Common themes discussed in the NYC and DC subreddits."}
)
output2 = {"question" : results2["input"], 
           "context": [r.metadata for r in results2["context"]], 
          "answer": results2["answer"]}

#print(results1["context"][0].metadata)
with open("problem3_task2.txt", "w") as f:
    json.dump(output2, f, indent=4)

In [32]:
doc_ids = list(vectorstore.index_to_docstore_id.values())

# Retrieve each document using the docstore's search method
docs = [vectorstore.docstore.search(doc_id) for doc_id in doc_ids]

retriever_vectordb = vectorstore.as_retriever(search_type="mmr", search_kwargs={"k": 4})
keyword_retriever = BM25Retriever.from_documents(docs)
ensemble_retriever = EnsembleRetriever(retrievers=[retriever_vectordb, keyword_retriever], weights=[0.5, 0.5])

rag_chain_e = create_retrieval_chain(ensemble_retriever, question_answer_chain)

results3 = rag_chain_e.invoke(
    {"input": "Write a newspaper article on how do creative people sustain themselves in NYC and is same as what creative people in DC do. Quote relevant posts from reddit. Sign the article by a made up name that is based on the data in the subreddits"}
)

output3 = {"question" : results3["input"], 
           "context": [r.metadata for r in results3["context"]], 
          "answer": results3["answer"]}

#print(results1["context"][0].metadata)
with open("problem3_task3.txt", "w") as f:
    json.dump(output3, f, indent=4)